In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
tf.__version__

In [ ]:
train_dir = 'dataset_balanceado_final/train'
validation_dir = 'dataset_balanceado_final/validation'
test_dir = 'dataset_balanceado_final/test'

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

IMG_SIZE = 150
BATCH_SIZE = 32

In [ ]:
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical' # categorical porque temos várias classes, senão seria binário (2 classes)
)

In [ ]:
validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

In [ ]:
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

In [ ]:
data_augmentation = keras.Sequential([layers.RandomFlip("horizontal"), # Efeito "espelho" a partir do eixo horizontal
                                      layers.RandomFlip("vertical"),# Aplica inversão vertical nas imagens.
                                      layers.RandomTranslation(0.1, 0.2), 
                                      layers.RandomRotation(0.4),])

In [ ]:
# Utiliza uma função(do sckicit-learn) para avaliar o desempenho do modelo, indicando Métricas como: 
    # f1-score do modelo
    # accuracy do modelo
    # accuracy por classe 

from sklearn.metrics import classification_report
import numpy as np

def print_classification_metrics(model, dataset, phase_name):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        preds = model.predict(images)
        y_true.extend(np.argmax(labels.numpy(), axis=1))
        y_pred.extend(np.argmax(preds, axis=1))

    print(f"\n {phase_name}")
    print(classification_report(y_true, y_pred, digits=4))


In [ ]:
# Define a camada de entrada do modelo com o formato das imagens (altura, largura, 3 canais RGB).
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)) 

In [ ]:
#Aplica as transformações aleatórias às imagens (definidas no inicio do notebook) para aumentar a diversidade do dataset de treino e evitar overfitting.
x = data_augmentation(inputs) 

In [ ]:
# Normaliza os valores dos pixels das imagens de entrada para o intervalo [0, 1].
x = layers.Rescaling(1./255)(x) 

In [ ]:
# Primeira camada convolucional com 64 filtros 3x3 e ativação ReLU
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding="same")(x) 
# adicionado o padding="same" para garantir que o output da camada convolucional tem a mesma dimensão espacial (altura e largura) que o input, após a operação de convolução.

In [ ]:
# Primeira camada de pooling máximo (2x2) para reduzir a dimensionalidade.
x = layers.MaxPooling2D(pool_size=2)(x)

In [ ]:
# Segunda camada convolucional com 128 filtros 3x3 e ativação ReLU.
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same")(x)
# Segunda camada de pooling máximo (2x2).
x = layers.MaxPooling2D(pool_size=2)(x)

In [ ]:
# Terceira camada convolucional com 128 filtros 3x3 e ativação ReLU.
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same")(x)
# Terceira camada de pooling máximo (2x2).
x = layers.MaxPooling2D(pool_size=2)(x)

In [ ]:
# Achata o output das camadas convolucionais para um vetor 1D.
x = layers.Flatten()(x)

In [ ]:
# tirar comentario desta linha abaixo se queremos usar batch normalization
# porem foi testado varias vezes e em diferentes camadas da rede, mas não melhorou o desempenho do modelo (e o optuna tambem sugeriu nao utilizar)
#x = layers.BatchNormalization( axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer="zeros",    moving_mean_initializer="zeros", moving_variance_initializer="ones")(x)

In [ ]:
# Aplica Dropout (50%) para desativar aleatoriamente neurónios, prevenindo o overfitting.
# Neste caso, para este modelo em especifico, atingiu a pior acc sem dropout
#x = layers.Dropout(0.5)(x)

In [ ]:
# Definir funçao de Regularização L2 (opcional)
reg = regularizers.l2(0.01)  # Executar para ativar

# Camada densa (totalmente conectada) com 128 neurónios, ativação ReLU e funçao de regularizaçao L2
x = layers.Dense(128, activation="relu", kernel_regularizer=reg)(x)

In [ ]:
# Camada de saída densa com 7 neurónios e ativação Softmax (para classificação categórica).
outputs = layers.Dense(7, activation="softmax")(x) 

In [ ]:
# Cria o modelo Keras usando as camadas de entrada e saída definidas.
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# Configura o otimizador: SGD
# Define a função de loss: MSE
# Indica que a 'accuracy' (precisão) será a métrica durante o treino.
model.compile(
    optimizer=tf.keras.optimizers.SGD(),
    loss='mse',
    metrics=['accuracy'])

In [ ]:
#model.compile(
    #optimizer='SGD', 
    #loss='mse',
    #metrics=['accuracy'])

In [ ]:
#model.compile(
    #optimizer=tf.keras.optimizers.RMSprop(),
    #loss=tf.keras.losses.KLDivergence(),
    #metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset, #Inicia o treino do modelo usando o conjunto de dados de treino.
    epochs=25,     # O modelo será treinado por 25 épocas (passagens completas pelo conjunto de treino).
    validation_data=validation_dataset) # Usa o conjunto de dados de validação para monitorizar o desempenho do modelo em dados não vistos durante o treino.

In [ ]:
print_classification_metrics(model, test_dataset, "Modelo 1 : CNN de raiz")

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Obter predições no test_dataset
y_true = []
y_pred = []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

class_names = test_dataset.class_names  

# Criar e mostrar a matriz de confusão
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45, values_format='d')
plt.title("Matriz de Confusão - Teste")
plt.tight_layout()
plt.show()

In [ ]:
model.save("modelS_2B3_com_data_aug_SGD_MSE_worst_acc.keras")